In [152]:
import numpy as np
import pandas as pd
import os



In [153]:
lista_de_arquivos = os.listdir("/home/otavio/Área de Trabalho/Sara_dataset")

In [154]:
label_map = {
        "Situação referencia index": 0,
        "Situação rolo 3 da esquerda levantado index": 1,
        "Situação rolo 3 da esquerda removido index": 2,
        "Situação rolo 3 da direita levantado index": 3,
        "Situação rolo 3 da direita removido index": 4,
    }



In [155]:
from aeon.classification.feature_based import Catch22Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from aeon.transformations.collection.convolution_based import Rocket

In [156]:
casos = []
labels = []


In [157]:
for i in lista_de_arquivos:
    for j in label_map:
        if(j in i):
            labels.append(label_map[j])
            df = pd.read_csv(f"Sara_dataset/{i}")
            X = df.drop(df.columns[0], axis=1)
            X = X['Gyroscope X']
            casos.append(X)


            
                        

In [158]:
X = casos
y = np.array(labels)

# Rocket com RidgeClassifier

In [159]:
from sklearn.model_selection import train_test_split
from aeon.classification.feature_based import Catch22Classifier
from sklearn import metrics

X_treino, X_teste, y_treino, y_teste = train_test_split(np.array(X), np.array(y), test_size=0.25, random_state=42)

In [160]:
X_treino

array([[-28.437968  , -34.90919346, -41.10916122, ...,  86.77745659,
         86.6040299 ,  85.502058  ],
       [-33.7077274 , -34.2162225 , -36.51786125, ...,  67.37559376,
         69.33368083,  74.076028  ],
       [ 20.4467852 ,  14.80748578,   7.13038686, ...,  74.22119057,
         65.41559506,  57.827586  ],
       ...,
       [ 10.775334  ,  10.65627107,   4.02700196, ..., 103.67871886,
        104.94048833, 104.313518  ],
       [-24.5428164 , -26.33159123, -28.62877682, ...,  83.29349686,
         85.08843237,  87.24251   ],
       [ 40.96844   ,  40.46721463,  35.30438515, ...,  58.0098162 ,
         54.22432219,  54.06896   ]])

In [161]:
rocket = Rocket(num_kernels=512)  # by default, ROCKET uses 10,000 kernels
rocket.fit(X_treino)
X_train_transform = rocket.transform(X_treino)


In [162]:
from sklearn.linear_model import RidgeClassifierCV

In [163]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
classifier.fit(X_train_transform, y_treino)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

In [164]:
X_test_transform = rocket.transform(X_teste)

In [165]:
classifier.score(X_test_transform, y_teste)

0.717948717948718

# Rocket + Isolation Forest

In [166]:
from sklearn.ensemble import IsolationForest



In [167]:
casos = np.array(casos)
labels = np.array(labels)

In [168]:
reference_list = []
for i in y:
  if(i == 0):
    reference_list.append(True)
  else:
    reference_list.append(False)

reference_list = np.array(reference_list)


# Separando entre treinamento e teste
X_train = np.array(casos[reference_list])
X_test = np.array(casos[np.invert(reference_list)])
y_train = labels[reference_list]
y_test = labels[np.invert(reference_list)]

In [169]:
rocket = Rocket(num_kernels=512)  # by default, ROCKET uses 10,000 kernels
rocket.fit(X_train)
X_train_transform = rocket.transform(X_train)
X_test_transform = rocket.transform(X_test)


In [170]:
# Treinando o modelo
iForest = IsolationForest(max_samples = len(X_train), random_state = 42) #Informando parâmetros do modelo

iForest.fit(X_train) #Comando que treina o Modelo

# Predições para o treinamento e teste

y_pred_train1 = iForest.predict(X_train)
y_pred_test1 = iForest.predict(X_test)

y_pred_total = iForest.predict(X)

In [171]:
print(y)

[4 0 1 1 0 0 0 4 3 1 0 0 4 3 4 3 4 4 1 4 1 4 1 3 4 3 4 1 2 2 1 0 0 0 1 0 3
 4 3 3 2 2 3 3 1 2 1 0 1 2 3 2 3 0 1 0 0 3 4 4 1 2 0 2 2 0 0 2 2 4 1 2 0 0
 2 4 2 4 1 3 1 0 3 1 4 1 1 0 2 4 4 1 3 3 3 1 3 4 2 0 3 4 4 3 0 4 3 4 1 2 1
 2 3 4 3 1 2 2 4 2 3 1 2 1 0 0 4 2 2 2 0 2 2 4 4 4 1 2 2 2 0 3 3 2 4 4 1 3
 3 3 3 3 1 2 3 4]


In [172]:
k = 0
for i in y_pred_total:
    for j in y:
        if(i == -1 and j != 0):
            k = k + 1
        elif(i == 1 and j == 0):
            k = k + 1

In [173]:
acc = k / len(y)
print(acc)

78.0
